In [3]:
import os
from pathlib import Path
from tempfile import mkdtemp
from warnings import filterwarnings
from dotenv import load_dotenv
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

TypeError: str expected, not NoneType

In [34]:
os.getcwd()

'/Users/A200154990/Langchain/CSKBOT'

In [48]:
!pip install llama-index openai


I0000 00:00:1751518161.267915 3806843 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


In [10]:
!pip install -q --progress-bar off llama-index-readers-docling \
llama-index-node-parser-docling llama-index-embeddings-huggingface \
llama-index-llms-huggingface-api llama-index-vector-stores-milvus \
llama-index-readers-file

In [ ]:
from llama_index.readers.docling import DoclingReader
from llama_index.core import download_loader

In [6]:
reader = DoclingReader()

# Load the PDF file
documents = reader.load_data("Internal.pdf")

# Display output
for doc in documents:
    print(doc.text[:5000])  # Print first 500 chars

## Internal IT Administration: Questions and Answers

This document provides a comprehensive list of internal IT administration questions across software, hardware, networking, and VPN domains, with detailed answers. The content is structured for easy indexing and retrieval, and includes various formats (paragraphs, lists, tables, JSON) to challenge a RAG pipeline.

## Software Administration

- Question: How do you schedule and manage updates for software applications across multiple enterprise workstations? 1.

Answer: Use centralized   patch   management   tools   to   automate   deployment.   For   example, configure Windows Server Update Services (WSUS) or System Center Configuration Manager (SCCM) for Windows machines, and tools like Ansible, Puppet, or Chef for Linux environments. Create deployment groups (e.g., by department or operating system) and schedule updates during   off-peak   hours   to   minimize   disruption.   Always   test   updates   in   a   staging   or   pilot

In [5]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.llms.openai import OpenAI


In [6]:
#""
'''BAAI/bge-small-en-v1.5 - Beijing Academy of Artificial Intelligence (BAAI)
Sentence Embedding Model / Text Embedding Model
Specifically designed for English text.
Based on E5 architecture, which itself is a modification of the MiniLM 
(or similar lightweight Transformer) architecture optimized for embedding tasks.
~60 million parameters'''

EMBED_MODEL = HuggingFaceEmbedding("BAAI/bge-small-en-v1.5")


embed_dim = len(EMBED_MODEL.get_text_embedding("hi"))#

In [7]:
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.readers.docling import DoclingReader
from llama_index.vector_stores.milvus import MilvusVectorStore

In [8]:
SOURCE = 'Internal.pdf'
#SOURCE = "https://arxiv.org/pdf/2408.09869"

In [9]:
reader = DoclingReader()
node_parser = MarkdownNodeParser()
MILVUS_URI = str(Path(mkdtemp())/ 'docling_ahtsham.db')
vector_store = MilvusVectorStore(uri=MILVUS_URI,dim=embed_dim,overwrite=True)
index = VectorStoreIndex.from_documents(
    documents=reader.load_data(SOURCE),
    transformations=[node_parser],
    storage_context=StorageContext.from_defaults(vector_store=vector_store),
    embed_model=EMBED_MODEL,
)


/opt/anaconda3/envs/langchain_env/lib/python3.10/site-packages/milvus_lite/__init__.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2025-07-03 15:20:13,943 [DEBUG][_create_connection]: Created new connection using: 20ebdbc230994adb9ec652df066b91f4 (async_milvus_client.py:599)


In [19]:
models = ['gpt-3.5-turbo','text-davinci-003']

llm = OpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    max_tokens=512
)



In [20]:


QUERY =  'How do you ensure software compliance (licensing) in an organization '
result = index.as_query_engine(llm=llm).query(QUERY)
print(f"Q: {QUERY}\nA: {result.response.strip()}\n\nSources:")
display([(n.text, n.metadata) for n in result.source_nodes])

Retrying llama_index.llms.openai.base.OpenAI._chat in 0.8821814833687962 seconds as it raised APIConnectionError: Connection error..
Retrying llama_index.llms.openai.base.OpenAI._chat in 0.11924063986921829 seconds as it raised APIConnectionError: Connection error..


APIConnectionError: Connection error.

In [26]:


QUERY =  'How old was albert einstein when he died?'
result = index.as_query_engine(llm=GEN_MODEL).query(QUERY)
print(f"Q: {QUERY}\nA: {result.response.strip()}\n\nSources:")
display([(n.text, n.metadata) for n in result.source_nodes])